# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Deploy de Modelos de Machine Learning</font>

# <font color='blue'>Amazon SageMaker</font>
## <font color='blue'>Lab</font>
### <font color='blue'>Deploy de Modelo Para Previsão de Doenças Usando Regsitros Médicos Eletrônicos</font>

## Parte 5 - Batch Transformation

O Batch Tranformation é a divisão dos dados em partes menores a fim de evitar estouro da memória quando treinamos modelos de Machine Learning.

Aqui você tem um exemplo completo de aplicar esse recurso usando o SageMaker. Vamos usar o modelo treinado e apenas ajustá-lo com Batch Transformation para fazer as previsões.

In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

## Imports 

In [ ]:
import time
import boto3
import sagemaker
import pandas as pd
from sagemaker import get_execution_role
from time import gmtime, strftime

## Carrega os Dados e Define os Parâmetros

In [ ]:
# Parâmetros
session = boto3.Session()
sagemaker_execution_role = get_execution_role()
sagemaker_session = sagemaker.session.Session()
sagemaker_client = boto3.client('sagemaker', region_name = session.region_name)
s3_client = boto3.client('s3')

In [ ]:
# Altere para o nome do seu bucket
s3_bucket = 'dsa-deploy-app'
prefix = 'dados'

In [ ]:
batch_input = f's3://{s3_bucket}/{prefix}/'
batch_input

In [ ]:
batch_output = f's3://{s3_bucket}/{prefix}/'
batch_output

In [ ]:
current_timestamp = strftime("%Y-%m-%d-%H-%M-%S", gmtime())

In [ ]:
TRAINING_JOB_NAME = 'classifier-2021-04-01-17-02-19-375'  
MODEL_NAME = f'modelo-xgboost-model-{current_timestamp}'
BATCH_JOB_NAME = f'modelo-xgboost-batch-job-{current_timestamp}'

## Criando o Modelo

In [ ]:
# Image URI
container_uri = sagemaker.image_uris.retrieve(region = session.region_name, 
                                              framework = 'xgboost', 
                                              version = '1.0-1', 
                                              image_scope = 'training')

In [ ]:
# Info sobre o job de treinamento
info = sagemaker_client.describe_training_job(TrainingJobName = TRAINING_JOB_NAME)
info

In [ ]:
# Artefatos do modelo
model_artifact_url = info['ModelArtifacts']['S3ModelArtifacts']
model_artifact_url

In [ ]:
# Container primário
primary_container = {'Image': container_uri, 'ModelDataUrl': model_artifact_url}

In [ ]:
# Criação do modelo
response = sagemaker_client.create_model(ModelName = MODEL_NAME,
                                         ExecutionRoleArn = sagemaker_execution_role,
                                         PrimaryContainer = primary_container)

In [ ]:
response

## Batch Transformer Para Inferência

In [ ]:
# Request com a configuração para executar o job
request = {
    "TransformJobName": BATCH_JOB_NAME,
    "ModelName": MODEL_NAME,
    "BatchStrategy": "MultiRecord",
    "TransformOutput": {
        "S3OutputPath": batch_output
    },
    "TransformInput": {
        "DataSource": {
            "S3DataSource": {
                "S3DataType": "S3Prefix",
                "S3Uri": batch_input 
            }
        },
        "ContentType": "text/csv",
        "SplitType": "Line",
        "CompressionType": "None"
    },
    "TransformResources": {
            "InstanceType": "ml.m5.xlarge",
            "InstanceCount": 1
    }
}

In [ ]:
# Cria o job
response = sagemaker_client.create_transform_job(**request)
response

In [ ]:
while(True):
    response = sagemaker_client.describe_transform_job(TransformJobName = BATCH_JOB_NAME)
    status = response['TransformJobStatus']
    if  status == 'Completed':
        print("Job finalizado com status: {}".format(status))
        break
    if status == 'Failed':
        message = response['FailureReason']
        print('O job falhou com o seguinte erro: {}'.format(message))
        raise Exception('Transform job failed') 
    print("Status atual do job: {}".format(status))    
    time.sleep(30) 

## Avaliação

In [ ]:
key = f'{prefix}/batch_test.csv.out'

In [ ]:
obj = s3_client.get_object(Bucket = s3_bucket, Key = key)
results_df = pd.read_csv(obj['Body'], names = ['Predictions'])

In [ ]:
results_df

# Fim